In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
from process_raw_data import *

# 6060 term

In [3]:
import pandas as pd
df = pd.read_csv("raw_data/6060_terminology_glossary.csv")
df = df[['english', 'chinese', 'arabic', 'french', 'russian', 'japanese']]

In [4]:
# !python -m spacy download en_core_web_trf

In [5]:
import spacy
nlp = spacy.load("en_core_web_trf")
import en_core_web_trf
nlp = en_core_web_trf.load()


In [6]:
word_lst = df['english'].values.tolist()
pos_lst = []
for word in word_lst:
    doc = nlp(word)
    for token in doc:
        pos_lst.append(token.pos_)
        # print(token.pos_)
        # ADJ, NOUN, PROPN, VERB
len(pos_lst), len(word_lst)

(309, 309)

In [7]:
df['pos_tag'] = pos_lst
df['pos_tag'].value_counts()

pos_tag
NOUN     155
PROPN     68
VERB      37
ADJ       35
ADV       12
ADP        2
Name: count, dtype: int64

In [8]:
df.loc[df['english'] == 'question']

,english,chinese,arabic,french,russian,japanese,pos_tag
60,question,问题,سؤال,question,вопрос,質問,NOUN


In [9]:
df.loc[df['pos_tag'] == 'NOUN', 'english'].values.tolist()

['Modeling',
 'task',
 'dataset',
 'models',
 'words',
 'language',
 'languages',
 'grammar',
 'linguistics',
 'source',
 'vocabulary',
 'tasks',
 'text',
 'speech',
 'detection',
 'sentence',
 'spans',
 'texts',
 'word',
 'model',
 'news',
 'training',
 'tokens',
 'encoding',
 'encode',
 'token',
 'number',
 'features',
 'transformer',
 'transformers',
 'metrics',
 'question',
 'embeddings',
 'information',
 'resources',
 'analysis',
 'paper',
 'Morphology',
 'research',
 'tokenization',
 'algorithm',
 'subword',
 'morphemes',
 'representation',
 'algorithms',
 'compositionality',
 'input',
 'document',
 'verbs',
 'encoder',
 'vectors',
 'nouns',
 'pronouns',
 'inference',
 'decoding',
 'correlations',
 'attention',
 'computation',
 'prediction',
 'classification',
 'problem',
 'vector',
 'baselines',
 'gradient',
 'sequences',
 'data',
 'quality',
 'finetuning',
 'user',
 'evaluation',
 'translations',
 'translation',
 'regression',
 'ablation',
 'method',
 'answer',
 'relations',
 '

In [10]:
df = df.loc[~df['english'].isin(unrelated_terms['6060_initiative'])]

In [11]:
df.loc[df['english'].isin(to_lowercase_terms['6060_initiative']), 'english'] = df.loc[df['english'].isin(to_lowercase_terms['6060_initiative']), 'english'].apply(lambda x: x.lower())

In [12]:
df

,english,chinese,arabic,french,russian,japanese,pos_tag
1,modeling,造型,نمذجة,modélisation,моделирование,モデリング,NOUN
2,lexical,词汇,معجمي,lexicale,словарный,語彙的,ADJ
3,task,任务,مهمة,tâche,Задача,タスク,NOUN
4,dataset,数据集,مجموعة بيانات,données,набор данных,データセット,NOUN
5,models,模型,نماذج,modèles,Модели,モデル,NOUN
...,...,...,...,...,...,...,...
304,extraction,提取,استخراج,extraction,Извлечение,抽出,NOUN
305,XLNet,XLNet,XLNet,XLNet,XLNet,XLNet,PROPN
306,multitask,多任务,متعددة المهام,multi-tâches,Многозадачность,マルチタスク,PROPN
307,MTDNN,MTDNN,MTDNN,MTDNN,MTDNN,MTDNN,PROPN


In [14]:
def align_capitalization(w1, w2):
    w3 = ""
    if w1[0].isupper():
        w3 += w2[0].upper()
    else:
        w3 += w2[0].lower()
    w3 += w2[1:]
    return w3
df['french'] = df.apply(lambda row: align_capitalization(row['english'], row['french']), axis=1)
df['russian'] = df.apply(lambda row: align_capitalization(row['english'], row['russian']), axis=1)

In [22]:
df.to_csv("processed_data/6060_terminology_glossary.csv")

# wiki_ai & wiki_cs

In [41]:
import json
wiki_ai = json.load(open("raw_data/wiki_ai.json", 'r'))
wiki_cs = json.load(open("raw_data/wiki_cs.json", 'r'))

In [42]:
wiki_ai = [{"term": format_wiki_terms(item['term']), "explanation": item['explanation']} for item in wiki_ai]
wiki_cs = [{"term": format_wiki_terms(item['term']), "explanation": item['explanation']} for item in wiki_cs]

In [43]:
wiki_ai[0]

{'term': 'abductive logic programming',
 'explanation': 'A high-level knowledge-representation framework that can be used to solve problems declaratively based on abductive reasoning . It extends normal logic programming by allowing some predicates to be incompletely defined, declared as abducible predicates.'}

In [44]:
# translate the English term to other languages
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append("../translator")
sys.path.append("..")
from translate_term import translate_term
df = pd.DataFrame().from_records(wiki_ai)
for language in tqdm(["Arabic", "French", "Japanese", "Russian"]):
    translated_term = []
    for item in tqdm(wiki_ai):
        translated_term.append(translate_term(item['term'], language, explanation=item['explanation'], model='gpt-4o'))
    df[f"term_{language.lower()}"] = translated_term

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [11:16<00:00, 169.23s/it]


In [45]:
df.to_csv("processed_data/wiki_ai2.csv")

In [46]:
# translate the English term to other languages
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append("../translator")
sys.path.append("..")
from translate_term import translate_term
df = pd.DataFrame().from_records(wiki_cs)
for language in tqdm(["Chinese", "Arabic", "French", "Japanese", "Russian"]):
    translated_term = []
    for item in tqdm(wiki_cs):
        translated_term.append(translate_term(item['term'], language, explanation=item['explanation'], model='gpt-4o'))
    df[f"term_{language.lower()}"] = translated_term

100%|██████████| 5/5 [11:24<00:00, 136.85s/it]


In [47]:
df.to_csv("processed_data/wiki_cs.csv")

# aggregate term dictionary

In [60]:
wiki_ai = pd.read_csv("processed_data/wiki_ai.csv")
wiki_cs = pd.read_csv("processed_data/wiki_cs.csv")
initiative = pd.read_csv("processed_data/6060_terminology_glossary.csv")

In [61]:
df = pd.DataFrame()
langs = ['english', 'chinese', 'arabic', 'french', 'russian', 'japanese']
for lang in langs:
    lst = []
    lst.extend(initiative[lang].values.tolist())
    if lang == 'english':
        lst.extend(wiki_ai['term'].values.tolist())
        lst.extend(wiki_cs['term'].values.tolist())
    else:
        lst.extend(wiki_ai[f'term_{lang}'].values.tolist())
        lst.extend(wiki_cs[f'term_{lang}'].values.tolist())
    df[lang] = lst

In [62]:
df.drop_duplicates(inplace=True)

In [63]:
# check non-unique values
non_unique_vals = df['english'][df['english'].duplicated(keep=False)]
non_unique_rows = df[df['english'].isin(non_unique_vals)]
non_unique_rows

,english,chinese,arabic,french,russian,japanese
41,transformer,转换器,محول,conversion,преобразователь,トランスフォーマー
60,algorithm,算法,خوارزمية,algorithme,алгоритм,アリゴリズム
86,computation,计算,حساب,calcul,вычисление,計算
108,method,方法,طريقة,méthode,метод,方法
111,sequence,顺序,التسلسل,séquence,последовательность,シーケンス
122,parameter,参数,معلمة,paramètres,параметр,パラメータ
129,graph,图形,رسم بياني,"graphique, diagramme",диаграммa,グラフ
133,event,事件,حدث,activité,событие,イベント
140,heuristic,启发式,إرشادي,heuristique,эвристический,ヒューリスティック
158,syntax,句法,بناء جملة,syntaxe,синтаксис,構文論


In [64]:
terms = list(set(non_unique_rows['english'].values.tolist()))
len(terms)

29

In [65]:
terms

['abstract data type',
 'algorithm',
 'syntax',
 'computation',
 'computer vision',
 'robotics',
 'semantics',
 'machine vision',
 'concatenation',
 'heuristic',
 'cognitive science',
 'computer science',
 'node',
 'abstraction',
 'software',
 'data mining',
 'parameter',
 'event',
 'logic programming',
 'intelligent agent',
 'agent architecture',
 'machine learning',
 'open-source software',
 'sequence',
 'method',
 'domain',
 'transformer',
 'internet',
 'graph']

In [ ]:
df.to_csv("processed_data/merged_term_dict.csv")

In [ ]:
# next step: manually unify duplicated rows that are leftover

# get the "nouns only" dictionary

In [2]:
import pandas as pd
# import spacy
# nlp = spacy.load("en_core_web_trf")
import en_core_web_trf
nlp = en_core_web_trf.load()

In [3]:
df = pd.read_csv("processed_data/merged_term_dict.csv")
term_list = df['english'].values.tolist()

In [4]:
res_list = []
for term in term_list:
    res = nlp(term)
    res = [np for np in res.noun_chunks]
    res_list.append(res)
df['pos_tags'] = res_list

In [5]:
res_list = []
for term in term_list:
    res = nlp(term)
    for token in res:
        if token.pos_ in ['NOUN', 'PROPN']:
            res_list.append(token.pos_)
            break
    else:
        res_list.append("")
df['has_noun'] = res_list

In [33]:
df.loc[df['pos_tags'].apply(lambda x: x == [])]

,Unnamed: 0,english,chinese,arabic,french,russian,japanese,pos_tags,has_noun
1,1,lexical,词汇,معجمي,lexicale,словарный,語彙的,[],
7,7,linguistic,语言化,لغوي,linguistique,лингвистический,言語的,[],
14,14,downstream,下游,البيانات المنخفضة,downstream,подчиненная,下流,[],
16,16,parsing,解析,تحليل,analyse syntaxique,парсинг,構文解析,[],
20,20,extracting,提取,استخراج,extraction,извлечение,抽出,[],
...,...,...,...,...,...,...,...,...,...
772,818,peripheral,外设,محيطي,Périphérique,Периферийное устройство,周辺機器,[],
784,834,queue,队列,طابور,file d'attente,очередь,キュー,[],
807,860,set,集合,مجموعة,ensemble,Множество,集合,[],
840,895,upload,上传,رفع,Téléverser,загрузить,アップロード,[],


In [6]:
df.loc[df['has_noun'].apply(lambda x: x == "")]

,Unnamed: 0,english,chinese,arabic,french,russian,japanese,pos_tags,has_noun
1,1,lexical,词汇,معجمي,lexicale,словарный,語彙的,[],
7,7,linguistic,语言化,لغوي,linguistique,лингвистический,言語的,[],
14,14,downstream,下游,البيانات المنخفضة,downstream,подчиненная,下流,[],
16,16,parsing,解析,تحليل,analyse syntaxique,парсинг,構文解析,[],
20,20,extracting,提取,استخراج,extraction,извлечение,抽出,[],
...,...,...,...,...,...,...,...,...,...
772,818,peripheral,外设,محيطي,Périphérique,Периферийное устройство,周辺機器,[],
784,834,queue,队列,طابور,file d'attente,очередь,キュー,[],
807,860,set,集合,مجموعة,ensemble,Множество,集合,[],
840,895,upload,上传,رفع,Téléverser,загрузить,アップロード,[],


In [34]:
df.loc[df['pos_tags'].apply(lambda x: x == []) & ~df['has_noun'].apply(lambda x: x == "")]

,Unnamed: 0,english,chinese,arabic,french,russian,japanese,pos_tags,has_noun
264,276,AI-complete,AI 完备,مكتمل الذكاء الاصطناعي,AI-complet,AI-полный,AI完全,[],NOUN
360,375,data set,数据集,مجموعة بيانات,Ensemble de données,Набор данных,データセット,[],NOUN


In [35]:
df.loc[~df['pos_tags'].apply(lambda x: x == []) & df['has_noun'].apply(lambda x: x == "")]

,Unnamed: 0,english,chinese,arabic,french,russian,japanese,pos_tags,has_noun


In [7]:
df = df.loc[~df['has_noun'].apply(lambda x: x == "")][['english', 'chinese', 'arabic', 'french', 'russian', 'japanese']]

In [21]:
# lemmatize and remove plurals
import spacy
tokens = []
for idx, row in df.iterrows():
    doc = nlp(row['english'])
    new_token = "".join([token.lemma_ + token.whitespace_ for token in doc])
    tokens.append(new_token)
df['english_lemma'] = tokens

In [4]:
df = pd.read_csv("processed_data/merged_term_dict_noun_only.csv")

In [5]:
df1 = df[~df.duplicated(subset=['english_lemma'], keep=False)]
df2 = df[df.duplicated(subset=['english_lemma'], keep=False)]
df2 = df2[df2['english_lemma'] == df2['english']].drop_duplicates(subset=['english_lemma'])

In [8]:
df[df.duplicated(subset=['english_lemma'], keep=False)]

,Unnamed: 0,english,chinese,arabic,french,russian,japanese,english_lemma


In [7]:
new_df = pd.concat([df1, df2])
new_df.head(10)

,Unnamed: 0,english,chinese,arabic,french,russian,japanese,english_lemma
0,0,modeling,造型,نمذجة,modélisation,моделирование,モデリング,modeling
1,9,grammar,语法,قواعد,grammaire,грамматика,文法,grammar
2,10,linguistics,语言学,لغويات,linguistiques,языкознания,言語学,linguistic
3,11,source,来源,مصدر,source,источник,ソース,source
4,12,NLP,自然语言处理,معالجة اللغة الطبيعية,TAL traitement automatique du langage naturel,NLP,自然言語処理,NLP
5,13,vocabulary,词汇,مفردات,vocabulaire,словарь,語彙,vocabulary
6,18,speech,语音,الكلام,dicours,речь,スピーチ,speech
7,19,detection,检测,كشف,detéction,обнаружение,検出,detection
8,22,spans,跨度,امتدادات,étendage,пяди,スパン,span
9,25,CRF,CRF,CRF,CRF,CRF,CRF,CRF


In [30]:
new_df.to_csv("processed_data/merged_term_dict_noun_only.csv")

In [10]:
df[df['english_lemma'] != df['english']].head(50)

,Unnamed: 0,english,chinese,arabic,french,russian,japanese,english_lemma
2,10,linguistics,语言学,لغويات,linguistiques,языкознания,言語学,linguistic
8,22,spans,跨度,امتدادات,étendage,пяди,スパン,span
17,39,features,特征,ميزات,fonctions,функции,機能,feature
19,45,metrics,指标,مقاييس,indicateurs,метрики,メトリックス,metric
20,46,embeddings,嵌入,التضمينات,intégration,встраивания,埋込み,embedding
24,52,resources,资源,الموارد,ressources,ресурсы,リソース,resource
31,61,morphemes,语素,مورفيمات,morphèmes,морфем,形態素,morpheme
35,74,pronouns,代词,الضمائر,pronoms,местоимения,代名詞,pronoun
40,83,correlations,相关性,ارتباطات,corrélation,корреляции,相関,correlation
47,92,baselines,基线,خطوط الأساس,bases,базовых линий,ベースライン,baseline
